## Description

This notebook calculates areas of each crop type within given regions of interest.

In [ ]:
import os
import pickle
import json

import datacube
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
from datacube.utils import geometry
from deafrica_tools.spatial import xr_rasterize

## Load region boundaries and crop map

In [ ]:
admin_boundaries_file = (
    "data/Mozambique_boundary.shp"
)
area_of_interest_gdf = gpd.read_file(admin_boundaries_file).to_crs("EPSG:32736")

In [ ]:
crop_type_path='results/Test_areas_mosaic_croptype_merged_prediction.tif'
da_crop_type=rioxarray.open_rasterio(crop_type_path).squeeze()

In [ ]:
# Dictionary with class labels from previous step
labels_path = "results/class_labels.json"

# Read the class label dictionary
with open(labels_path, "r") as json_file:
    labels_dict = json.load(json_file)

## Calculate areas per polygon

In [ ]:
gdf_new=area_of_interest_gdf.copy()
for index, district in area_of_interest_gdf.iterrows():

    print(f"Processing polygon {index}")
    area_of_interest_gdf.loc[index,'ID']=index
    # Rasterize polygon
    district_mask = xr_rasterize(
        gdf=area_of_interest_gdf.iloc[[index]],
        da=da_crop_type,
        transform=da_crop_type.geobox.transform,
        crs="EPSG:32736",
    )
    for class_name, class_value in labels_dict.items():
        crop_type_area=np.sum(district_mask==class_value)/10000.0
        attr_name=class_name+"_area_km2"
        gdf_new.loc[index, attr_name] = crop_type_area
        print('area in km2 for {}: {}'.format(class_name,crop_type_area))

## Plot areas histogram

In [ ]:
gdf_new.plot(x='ID',y=[labels_dict.values()])

## Export to files

In [ ]:
gdf_new.to_file("data/crop_areas.geojson")

In [ ]:
# convert to CSV by dropping geometry
df = pd.DataFrame(gdf_new.drop(columns='geometry'))

df.to_csv("data/crop_areas.csv")